In [1]:
import warnings
warnings.filterwarnings('ignore')
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
import tensorflow as tf
from sklearn.preprocessing import OneHotEncoder, StandardScaler, FunctionTransformer
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()

Instructions for updating:
non-resource variables are not supported in the long term


# Problem 1 
## Looking back on scratch

(Example)

1. I had to initialize the weights

2. I needed an epoch loop

Let's now remember how they are implemented in the framework.

1. Need to implement Initializer class

2. Optimizer implementation required

3. It is necessary to implement the activation function

4. Requires mini-batch size split

5. Loss function needs to be defined and implemented

6. Need to implement forward / back processing for class of layers such as convolution layer, fully connected layer, etc.

# Problem 2
## Consider the correspondence between scratch and TensorFlow

In [2]:
"""
Binary classification of Iris dataset using neural network implemented in TensorFlow
"""
## Data set loading
dataset_path ="datasets_19_420_Iris.csv"
df = pd.read_csv(dataset_path)
## Condition extraction from data frame
df = df[(df["Species"] == "Iris-versicolor")|(df["Species"] == "Iris-virginica")]
y = df["Species"]
X = df.loc[:, ["SepalLengthCm", "SepalWidthCm", "PetalLengthCm", "PetalWidthCm"]]
y = np.array(y)
X = np.array(X)
## Convert labels to numbers
y[y=='Iris-versicolor'] = 0
y[y=='Iris-virginica'] = 1
y = y.astype(np.int)[:, np.newaxis]
## Divided into train and test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)
## Further divided into train and val
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=0)
class GetMiniBatch:
    """
    Iterator to get a mini-batch

     Parameters
     ----------
     X: ndarray, shape (n_samples, n_features) of the following form
       Training data
     y: ndarray, shape (n_samples, 1) of the following form
       Correct answer value
     batch_size: int
       Batch size
     seed: int
       NumPy random number seed
    """
    def __init__(self, X, y, batch_size = 10, seed=0):
        self.batch_size = batch_size
        np.random.seed(seed)
        shuffle_index = np.random.permutation(np.arange(X.shape[0]))
        self.X = X[shuffle_index]
        self.y = y[shuffle_index]
        self._stop = np.ceil(X.shape[0]/self.batch_size).astype(np.int)
    def __len__(self):
        return self._stop
    def __getitem__(self,item):
        p0 = item*self.batch_size
        p1 = item*self.batch_size + self.batch_size
        return self.X[p0:p1], self.y[p0:p1]        
    def __iter__(self):
        self._counter = 0
        return self
    def __next__(self):
        if self._counter >= self._stop:
            raise StopIteration()
        p0 = self._counter*self.batch_size
        p1 = self._counter*self.batch_size + self.batch_size
        self._counter += 1
        return self.X[p0:p1], self.y[p0:p1]
## Hyperparameter settings
learning_rate = 0.01
batch_size = 10
num_epochs = 10
n_hidden1 = 50
n_hidden2 = 100
n_input = X_train.shape[1]
n_samples = X_train.shape[0]
n_classes = 1
## Determine the shape of the arguments to pass to the calculation graph
X = tf.placeholder("float", [None, n_input])
Y = tf.placeholder("float", [None, n_classes])
## train mini batch iterator
get_mini_batch_train = GetMiniBatch(X_train, y_train, batch_size=batch_size)
def example_net(x):
    """
    Simple 3-layer neural network
    """
    ## Declaration of weight and bias
    weights = {
        'w1': tf.Variable(tf.random_normal([n_input, n_hidden1])),
        'w2': tf.Variable(tf.random_normal([n_hidden1, n_hidden2])),
        'w3': tf.Variable(tf.random_normal([n_hidden2, n_classes]))
    }
    biases = {
        'b1': tf.Variable(tf.random_normal([n_hidden1])),
        'b2': tf.Variable(tf.random_normal([n_hidden2])),
        'b3': tf.Variable(tf.random_normal([n_classes]))
    }
    layer_1 = tf.add(tf.matmul(x, weights['w1']), biases['b1'])
    layer_1 = tf.nn.relu(layer_1)
    layer_2 = tf.add(tf.matmul(layer_1, weights['w2']), biases['b2'])
    layer_2 = tf.nn.relu(layer_2)
    layer_output = tf.matmul(layer_2, weights['w3']) + biases['b3'] ## tf.add and + are equivalent
    return layer_output
## Read network structure                              
logits = example_net(X)
## Objective function
loss_op = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(labels=Y, logits=logits))
## Optimization method
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate)
train_op = optimizer.minimize(loss_op)
## Estimated result
correct_pred = tf.equal(tf.sign(Y - 0.5), tf.sign(tf.sigmoid(logits) - 0.5))
## Index value calculation
accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))
## Initialization of variable
init = tf.global_variables_initializer()

## Execution of calculation graph
with tf.Session() as sess:
    sess.run(init)
    for epoch in range(num_epochs):
        ## Loop for each epoch
        total_batch = np.ceil(X_train.shape[0]/batch_size).astype(np.int)
        total_loss = 0
        total_acc = 0
        for i, (mini_batch_x, mini_batch_y) in enumerate(get_mini_batch_train):
            ## Loop for each mini-batch
            sess.run(train_op, feed_dict={X: mini_batch_x, Y: mini_batch_y})
            loss, acc = sess.run([loss_op, accuracy], feed_dict={X: mini_batch_x, Y: mini_batch_y})
            total_loss += loss
            total_acc += acc
        total_loss /= n_samples
        total_acc /= n_samples
        val_loss, val_acc = sess.run([loss_op, accuracy], feed_dict={X: X_val, Y: y_val})
        print("Epoch {}, loss : {:.4f}, val_loss : {:.4f}, acc : {:.3f}, val_acc : {:.3f}".format(epoch, loss, val_loss, acc, val_acc))
    test_acc = sess.run(accuracy, feed_dict={X: X_test, Y: y_test})
    print("test_acc : {:.3f}".format(test_acc))


Epoch 0, loss : 0.8680, val_loss : 1.0526, acc : 0.750, val_acc : 0.750
Epoch 1, loss : 20.5905, val_loss : 9.4453, acc : 0.250, val_acc : 0.625
Epoch 2, loss : 5.7928, val_loss : 11.4675, acc : 0.750, val_acc : 0.375
Epoch 3, loss : 5.8330, val_loss : 3.6214, acc : 0.500, val_acc : 0.688
Epoch 4, loss : 0.9386, val_loss : 2.9975, acc : 0.750, val_acc : 0.688
Epoch 5, loss : 0.0101, val_loss : 1.2834, acc : 1.000, val_acc : 0.812
Epoch 6, loss : 0.0000, val_loss : 0.2076, acc : 1.000, val_acc : 0.938
Epoch 7, loss : 0.0000, val_loss : 0.0000, acc : 1.000, val_acc : 1.000
Epoch 8, loss : 0.0000, val_loss : 0.0043, acc : 1.000, val_acc : 1.000
Epoch 9, loss : 0.0000, val_loss : 0.0198, acc : 1.000, val_acc : 1.000
test_acc : 0.850


# Problem 3 
## Create a model of Iris using all three types of objective variables

In [3]:
## Pretreatment process
## Load dataset
dataset_path ="datasets_19_420_Iris.csv"
df = pd.read_csv(dataset_path)
## Condition extraction from data frame
y = df["Species"]
X = df.loc[:, ["SepalLengthCm", "SepalWidthCm", "PetalLengthCm", "PetalWidthCm"]]
y = np.array(y)
X = np.array(X)
## Convert labels to numbers
y[y=='Iris-setosa'] = 0
y[y=='Iris-versicolor'] = 1
y[y=='Iris-virginica'] = 2
y = y.astype(np.int)[:, np.newaxis]
## Divided into train and test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

## Normalization and one-hot
ss = StandardScaler().fit(X_train)
X_train = ss.transform(X_train)
X_test = ss.transform(X_test)

ec = OneHotEncoder(handle_unknown="ignore", sparse=False)
y_train = ec.fit_transform(y_train)
y_test = ec.fit_transform(y_test)

## Further divided into train and val
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=0)


## Setting various parameters
## Hyperparameter settings
learning_rate = 0.01
batch_size = 10
num_epochs = 10
n_hidden1 = 50
n_hidden2 = 100
n_input = X_train.shape[1]
n_samples = X_train.shape[0]
n_classes = 3
## Determine the shape of the arguments to pass to the calculation graph
X = tf.placeholder("float", [None, n_input])
Y = tf.placeholder("float", [None, n_classes])
## train mini batch iterator
get_mini_batch_train = GetMiniBatch(X_train, y_train, batch_size=batch_size)


## Network construction 
def example_net(x):
    """
    Simple 3-layer neural network
    """
    ## Initialization class definition
    initializer = tf.keras.initializers.he_normal()
    ## Declaration of weight and bias
    weights = {
        'w1': tf.Variable(initializer([n_input, n_hidden1])),
        'w2': tf.Variable(initializer([n_hidden1, n_hidden2])),
        'w3': tf.Variable(initializer([n_hidden2, n_classes]))
    }
    biases = {
        'b1': tf.Variable(initializer([n_hidden1])),
        'b2': tf.Variable(initializer([n_hidden2])),
        'b3': tf.Variable(initializer([n_classes]))
    }
    layer_1 = tf.add(tf.matmul(x, weights['w1']), biases['b1'])
    layer_1 = tf.nn.relu(layer_1)
    layer_2 = tf.add(tf.matmul(layer_1, weights['w2']), biases['b2'])
    layer_2 = tf.nn.relu(layer_2)
    layer_output = tf.matmul(layer_2, weights['w3']) + biases['b3'] ## tf.add and + are equivalent
    return layer_output

## Read network structure                               
logits = example_net(X)
## Objective function
loss_op = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=Y, logits=logits))
## Optimization method
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate)
train_op = optimizer.minimize(loss_op)
## Estimated result
correct_pred = tf.equal(tf.argmax(Y, 1), tf.argmax(logits, 1))
## Index value calculation
accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))
## Initialization of variable
init = tf.global_variables_initializer()


## Execution of calculation graph
with tf.Session() as sess:
    sess.run(init)
    for epoch in range(num_epochs):
        ## Loop for each epoch
        total_batch = np.ceil(X_train.shape[0]/batch_size).astype(np.int)
        total_loss = 0
        total_acc = 0
        for i, (mini_batch_x, mini_batch_y) in enumerate(get_mini_batch_train):
            ## Loop for each mini-batch
            sess.run(train_op, feed_dict={X: mini_batch_x, Y: mini_batch_y})
            loss, acc = sess.run([loss_op, accuracy], feed_dict={X: mini_batch_x, Y: mini_batch_y})
            total_loss += loss
            total_acc += acc
        total_loss /= n_samples
        total_acc /= n_samples
        val_loss, val_acc = sess.run([loss_op, accuracy], feed_dict={X: X_val, Y: y_val})
        print("Epoch {}, loss : {:.4f}, val_loss : {:.4f}, acc : {:.3f}, val_acc : {:.3f}".format(epoch, loss, val_loss, acc, val_acc))
    test_acc = sess.run(accuracy, feed_dict={X: X_test, Y: y_test})
    print("test_acc : {:.3f}".format(test_acc))


Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See `tf.nn.softmax_cross_entropy_with_logits_v2`.

Epoch 0, loss : 0.5979, val_loss : 0.7092, acc : 0.833, val_acc : 0.792
Epoch 1, loss : 0.0599, val_loss : 0.5213, acc : 1.000, val_acc : 0.750
Epoch 2, loss : 0.0624, val_loss : 0.7643, acc : 1.000, val_acc : 0.792
Epoch 3, loss : 0.0083, val_loss : 0.3113, acc : 1.000, val_acc : 0.833
Epoch 4, loss : 0.0146, val_loss : 0.3688, acc : 1.000, val_acc : 0.917
Epoch 5, loss : 0.0053, val_loss : 0.2729, acc : 1.000, val_acc : 0.917
Epoch 6, loss : 0.0043, val_loss : 0.2878, acc : 1.000, val_acc : 0.917
Epoch 7, loss : 0.0033, val_loss : 0.2723, acc : 1.000, val_acc : 0.917
Epoch 8, loss : 0.0027, val_loss : 0.2891, acc : 1.000, val_acc : 0.917
Epoch 9, loss : 0.0019, val_loss : 0.3010, acc : 1.000, val_acc : 0.917
test_acc : 1.000


# Problem 4
## Create a model of House Prices

# Data reading / preprocessing

In [4]:
## Data set loading
dataset_path ="train.csv"
df = pd.read_csv(dataset_path)

## Condition extraction from data frame
y = df["SalePrice"]
X = df.loc[:, ["GrLivArea", "YearBuilt"]]
y = np.array(y)
X = np.array(X)
y = y.astype(np.float)[:, np.newaxis]

## Divided into train and test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)
## Further divided into train and val
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=0)

## Normalization
x_scaler = StandardScaler()
x_scaler.fit(X_train)
X_train = x_scaler.transform(X_train)
X_val   = x_scaler.transform(X_val)
X_test  = x_scaler.transform(X_test)

## Logarithmic transformation
y_scaler = FunctionTransformer(func=np.log1p, inverse_func=np.expm1)
y_train = y_scaler.transform(y_train)
y_val   = y_scaler.transform(y_val)
y_test  = y_scaler.transform(y_test)


In [5]:
## Hyperparameter settings
learning_rate = 0.01
batch_size = 10
num_epochs = 10
n_hidden1 = 50
n_hidden2 = 100
n_input = X_train.shape[1]
n_samples = X_train.shape[0]
n_output = 1
## Determine the shape of the arguments to pass to the calculation graph
X = tf.placeholder(tf.float32, [None, n_input])
Y = tf.placeholder(tf.float32, [None,1])
## train mini batch iterator
get_mini_batch_train = GetMiniBatch(X_train, y_train, batch_size=batch_size)

# Network & Computation Graph Construction

In [6]:
def example_net(x):
    """
    Simple 3-layer neural network
    """
    initializer = tf.keras.initializers.he_normal()

    ## Declaration of Mito Bias
    weights = {
        'w1': tf.Variable(initializer([n_input, n_hidden1])),
        'w2': tf.Variable(initializer([n_hidden1, n_hidden2])),
        'w3': tf.Variable(initializer([n_hidden2, n_output]))
    }
    biases = {
        'b1': tf.Variable(initializer([n_hidden1])),
        'b2': tf.Variable(initializer([n_hidden2])),
        'b3': tf.Variable(initializer([n_output]))
    }
    layer_1 = tf.add(tf.matmul(x, weights['w1']), biases['b1'])
    layer_1 = tf.nn.relu(layer_1)
    layer_2 = tf.add(tf.matmul(layer_1, weights['w2']), biases['b2'])
    layer_2 = tf.nn.relu(layer_2)
    layer_output = tf.matmul(layer_2, weights['w3']) + biases['b3'] ## tf.add and + are equivalent
    return layer_output

## Read network structure                               
logits = example_net(X)

## Objective Function (MSE)
loss_op = tf.losses.mean_squared_error(labels=Y, predictions=logits)
## Optimization method
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate)
train_op = optimizer.minimize(loss_op)
## Index value calculation
rmse = tf.log(tf.reduce_mean(tf.square(tf.math.expm1(Y)  - tf.math.expm1(logits))))
## Initialization of variable
init = tf.global_variables_initializer()

# Execution of calculation graph

In [7]:
with tf.Session() as sess:
    sess.run(init)
    for epoch in range(num_epochs):
        ## Loop for each epoch
        total_batch = np.ceil(X_train.shape[0]/batch_size).astype(np.int)
        total_loss = 0
        total_score = 0
        for i, (mini_batch_x, mini_batch_y) in enumerate(get_mini_batch_train):
            ## Loop for each mini-batch
            sess.run(train_op, feed_dict={X: mini_batch_x, Y: mini_batch_y})
            loss, score = sess.run([loss_op, rmse], feed_dict={X: mini_batch_x, Y: mini_batch_y})
            total_loss += loss
            total_score += score
        total_loss /= n_samples
        total_score /= n_samples
        val_loss, val_score = sess.run([loss_op, rmse], feed_dict={X: X_val, Y: y_val})
        print("Epoch {}, loss : {:.4f}, val_loss : {:.4f}, acc : {:.3f}, val_acc : {:.3f}".format(epoch, loss, val_loss, score, val_score))
    test_score = sess.run(rmse, feed_dict={X: X_test, Y: y_test})
    print("test_score : {:.3f}".format(test_score))

Epoch 0, loss : 0.1937, val_loss : 0.5591, acc : 22.307, val_acc : 27.735
Epoch 1, loss : 0.0160, val_loss : 0.1134, acc : 19.786, val_acc : 22.489
Epoch 2, loss : 0.0078, val_loss : 0.0767, acc : 19.221, val_acc : 21.867
Epoch 3, loss : 0.0472, val_loss : 0.1285, acc : 21.444, val_acc : 22.781
Epoch 4, loss : 0.0537, val_loss : 0.1312, acc : 21.514, val_acc : 22.623
Epoch 5, loss : 0.0308, val_loss : 0.1032, acc : 20.831, val_acc : 22.072
Epoch 6, loss : 0.0200, val_loss : 0.0798, acc : 20.376, val_acc : 21.543
Epoch 7, loss : 0.0215, val_loss : 0.0776, acc : 20.453, val_acc : 21.507
Epoch 8, loss : 0.0241, val_loss : 0.0785, acc : 20.568, val_acc : 21.581
Epoch 9, loss : 0.0236, val_loss : 0.0750, acc : 20.570, val_acc : 21.583
test_score : 22.523


# Problem 5
## Create a model of MNIST

# Data reading / preprocessing

In [8]:
(X_train, y_train), (X_test, y_test) = tf.keras.datasets.mnist.load_data()

In [9]:
## Size specification
img_heigt = 28
img_width = 28
feature_size = int(img_heigt * img_width)

## Dimensionality reduction
X_train = X_train.reshape(-1, feature_size)
X_test  = X_test.reshape(-1, feature_size)

## Type conversion for normalization
X_train = X_train.astype(np.float32)
X_test  = X_test.astype(np.float32)

## Normalization
X_train /= 255
X_test /= 255

## One hot conversion
y_train = tf.keras.utils.to_categorical(y_train)
y_test = tf.keras.utils.to_categorical(y_test)

X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.1, random_state=0)

print("X shape:", X_train.shape)
print("y shape:", y_train.shape)

X shape: (54000, 784)
y shape: (54000, 10)


Setting of various parameters

In [10]:
learning_rate = 0.01 ## Learning rate
batch_size = 2700 ## Batch size
num_epochs = 10  ## Number of epochs
n_hidden1 = 256  ## Hidden layer 1
n_hidden2 = 128  ## Hidden layer 2
n_input = X_train.shape[1]
n_samples = X_train.shape[0]
n_output = y_train.shape[1]
## Determine the shape of the arguments to pass to the calculation graph
X = tf.placeholder(tf.float32, [None, n_input])
Y = tf.placeholder(tf.float32, [None, n_output])
## train mini batch iterator
get_mini_batch_train = GetMiniBatch(X_train, y_train, batch_size=batch_size)

In [11]:
def network(x):
  initializer = tf.keras.initializers.he_normal()
  ## Declaration of weight and bias
  weights = {
      "w1": tf.Variable(initializer([n_input, n_hidden1])),
      "w2": tf.Variable(initializer([n_hidden1, n_hidden2])),
      "w3": tf.Variable(initializer([n_hidden2, n_output]))
  }

  biases = {
      "b1": tf.Variable(initializer([n_hidden1,])),
      "b2": tf.Variable(initializer([n_hidden2,])),
      "b3": tf.Variable(initializer([n_output,]))
  }

  ## Forward propagation
  layer_1 = tf.matmul(x, weights["w1"]) + biases["b1"]
  layer_1 = tf.nn.relu(layer_1)
  layer_2 = tf.matmul(layer_1, weights["w2"]) + biases["b2"]
  layer_2 = tf.nn.relu(layer_2)
  layer_output = tf.matmul(layer_2, weights["w3"]) + biases["b3"]
  return layer_output

logits = network(X)

## Loss function
loss_op = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=Y, logits=logits))

## optimization
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate)
train_op = optimizer.minimize(loss_op)

## Estimated result
correct_pred = tf.equal(tf.argmax(Y, 1), tf.argmax(logits, 1))

## Index value calculation
accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))

## Initialization of variable
init = tf.global_variables_initializer()

In [12]:
with tf.Session() as sess:
    sess.run(init)
    for epoch in range(num_epochs):
        ## Loop for each epoch
        total_batch = np.ceil(X_train.shape[0]/batch_size).astype(np.int)
        total_loss = 0
        total_acc = 0
        for i, (mini_batch_x, mini_batch_y) in enumerate(get_mini_batch_train):
            ## Loop for each mini-batch
            sess.run(train_op, feed_dict={X: mini_batch_x, Y: mini_batch_y})
            loss, acc = sess.run([loss_op, accuracy], feed_dict={X: mini_batch_x, Y: mini_batch_y})
            total_loss += loss
            total_acc += acc
        total_loss /= n_samples
        total_acc /= n_samples
        val_loss, val_acc = sess.run([loss_op, accuracy], feed_dict={X: X_val, Y: y_val})
        print("Epoch {}, loss : {:.4f}, val_loss : {:.4f}, acc : {:.3f}, val_acc : {:.3f}".format(epoch, loss, val_loss, acc, val_acc))
    test_acc = sess.run(accuracy, feed_dict={X: X_test, Y: y_test})
    print("test_acc : {:.3f}".format(test_acc))

Epoch 0, loss : 0.3042, val_loss : 0.2695, acc : 0.906, val_acc : 0.916
Epoch 1, loss : 0.1810, val_loss : 0.1605, acc : 0.945, val_acc : 0.951
Epoch 2, loss : 0.1195, val_loss : 0.1181, acc : 0.961, val_acc : 0.964
Epoch 3, loss : 0.0868, val_loss : 0.0981, acc : 0.970, val_acc : 0.970
Epoch 4, loss : 0.0628, val_loss : 0.0874, acc : 0.983, val_acc : 0.975
Epoch 5, loss : 0.0474, val_loss : 0.0854, acc : 0.985, val_acc : 0.975
Epoch 6, loss : 0.0367, val_loss : 0.0807, acc : 0.990, val_acc : 0.976
Epoch 7, loss : 0.0354, val_loss : 0.0849, acc : 0.989, val_acc : 0.975
Epoch 8, loss : 0.0309, val_loss : 0.0948, acc : 0.991, val_acc : 0.973
Epoch 9, loss : 0.0302, val_loss : 0.0920, acc : 0.990, val_acc : 0.973
test_acc : 0.974
